here, i i will attempt to find the bug that is preventing certain protein mutations from running quickly...

In [12]:
import simtk.openmm as openmm
from openmmtools.constants import kB
import simtk.unit as unit
temperature = 300 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
from perses.rjmc.topology_proposal import TopologyProposal, NetworkXMolecule
from openmmtools.states import ThermodynamicState, SamplerState, CompoundThermodynamicState
from openmmtools import mcmc, utils
import openmmtools.cache as cache
from perses.dispersed.utils import configure_platform
#cache.global_context_cache.platform = configure_platform(utils.get_fastest_platform().getName())
from perses.annihilation.lambda_protocol import LambdaProtocol
from perses.annihilation.lambda_protocol import RelativeAlchemicalState, LambdaProtocol
import openmmtools.integrators as integrators
# from protein_test_cycle import *
import numpy as np

In [4]:
print(cache.global_context_cache.platform.getName())
#print(cache.global_context_cache.platform.getPropertyDefaultValue('CudaPrecision'))

CPU


In [5]:
def deserialize(filename):
    import pickle
    with open(filename, 'rb') as f:
        htf = pickle.load(f)
    return htf



In [22]:
fast_htf = deserialize('ALA_SER.vac.pkl')
slow_htf = deserialize('ALA_THR.vac.pkl') 
# fast_hss, _ = create_hss(f"fast_debug.nc", fast_htf)
# slow_hss, _ = create_hss(f"slow_debug.nc", slow_htf)

FileNotFoundError: [Errno 2] No such file or directory: 'ALA_THR.vac.pkl'

In [7]:
import copy

In [8]:
def create_integrator(htf, constraint_tol):
    """
    create lambda alchemical states, thermodynamic states, sampler states, integrator, and return context, thermostate, sampler_state, integrator
    """
    fast_lambda_alchemical_state = RelativeAlchemicalState.from_system(htf.hybrid_system)
    fast_lambda_alchemical_state.set_alchemical_parameters(0.0, LambdaProtocol(functions = 'default'))
    
    fast_thermodynamic_state = CompoundThermodynamicState(ThermodynamicState(htf.hybrid_system, temperature = temperature),composable_states = [fast_lambda_alchemical_state])
    
    fast_sampler_state = SamplerState(positions = htf._hybrid_positions, box_vectors = htf.hybrid_system.getDefaultPeriodicBoxVectors())
    
    integrator_1 = integrators.LangevinIntegrator(temperature = temperature,
                                                     timestep = 4.0* unit.femtoseconds,
                                                     splitting = 'V R O R V',
                                                     measure_shadow_work = False,
                                                     measure_heat = False,
                                                     constraint_tolerance = constraint_tol,
                                                     collision_rate = 5.0 / unit.picoseconds)
    
    fast_context, fast_integrator = cache.global_context_cache.get_context(fast_thermodynamic_state, integrator_1)
    
    
    fast_sampler_state.apply_to_context(fast_context)
    
    return fast_context, fast_thermodynamic_state, fast_sampler_state, fast_integrator


    
    
    
    

In [17]:
_, _, _, fast_int = create_integrator(fast_htf, 1e-6)
_, _, _, slow_int = create_integrator(slow_htf, 1e-6)
_, _, _, slow_int_mod1 = create_integrator(copy.deepcopy(slow_htf), 1e-5)
_, _, _, slow_int_mod2 = create_integrator(copy.deepcopy(slow_htf), 1e-4)
_, _, _, slow_int_mod3 = create_integrator(copy.deepcopy(slow_htf), 1e-3)

In [18]:
def time_integrator(integrator, num_steps = 10):
    import time
    _time = []
    integrator.step(1)
    for i in range(num_steps):
        start = time.time()
        integrator.step(1)
        end = time.time() - start
        _time.append(end)
    return np.array(_time)

In [19]:
fast = time_integrator(fast_int, num_steps = 100)
slow = time_integrator(slow_int, num_steps = 100)
slow_mod1 = time_integrator(slow_int_mod1, num_steps = 100)
slow_mod2 = time_integrator(slow_int_mod2, num_steps = 100)
slow_mod3 = time_integrator(slow_int_mod3, num_steps = 100)

In [20]:
print(np.average(fast), np.std(fast))
print(np.average(slow), np.std(slow))
print(np.average(slow_mod1), np.std(slow_mod1))
print(np.average(slow_mod2), np.std(slow_mod2))
print(np.average(slow_mod3), np.std(slow_mod3))
#print(np.average(slow_mod), np.std(slow_mod))

0.0012963438034057618 0.00017365259597791006
0.0012608838081359863 0.0001394109475754756
0.0012266659736633301 0.00011619802694094898
0.001241781711578369 0.00011578275430538863
0.0012153458595275878 0.00011124351557612184


^^^^^^^ perhaps if we remove the constraints, it will run faster? and lo, it does




## now, we have to figure out how we can fix this issue; let's up the constraint tolerance iteratively...


i'll iterate over a few orders of magnitude with the constraint tolerance to try to pull up the speed of integration...

In [ ]:
for tol in [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]:
    _, _, _, slow_int = create_integrator(fast_htf, tol)
    